In [1]:
import polars as pl
import os

In [2]:
PATH = os.getcwd()
path = f"{PATH}/../data/silver/"

/home/phillip/repo/Norgesnett/notebook/../data/silver/


In [52]:
# load features table
df = pl.read_parquet(os.path.join(path,'features'))

In [3]:
df = pl.read_parquet(os.path.join(path,'S_17279_T_1513'))

In [53]:
# Filter neighborhoods on plusskunde penetration
def r1_filter_penetration(df: pl.DataFrame, lower_limit: float):
    return df.filter(pl.col('res_pen_pers')>lower_limit)

In [13]:
df.select(pl.col('p_prod_kwh').max())
df.select(pl.col('meteringPointId').first()).item()
df.filter(pl.col('meteringPointId')=='707057500017299122').max()

fromTime,toTime,topology,meteringPointId,p_load_kwh,p_prod_kwh
datetime[μs],datetime[μs],str,str,f64,f64
2023-09-01 01:00:00,2023-09-01 02:00:00,"""S_17279_T_1513…","""70705750001729…",5.42,2.58


In [8]:
df.columns

['fromTime',
 'toTime',
 'topology',
 'meteringPointId',
 'p_load_kwh',
 'p_prod_kwh']

In [54]:
# Filter for neighborhood size (closed-set of AMI grouping having at least 1 PCC)
# Reference: https://gridarchitecture.pnnl.gov/media/Modern-Distribution-Grid_Volume_II_v2_0.pdf (20 -150 reference)
def r2_filter_ami_cnt_lower_limit(df: pl.DataFrame, lower_limit: float):
    return df.filter(pl.col('ami_cnt')>lower_limit)

In [55]:
# Filter for a neighborhood having a minimal number of production points
def r3_filter_ami_prod_cnt_lower_limit(df: pl.DataFrame, lower_limit: float):
    return df.filter(pl.col('ami_prod_cnt')>lower_limit)

In [56]:
# Filter for a neighborhood having at least 1 production point producing more than limit
def r4_filter_include_for_production_over(df: pl.DataFrame, lower_limit: float):
    return df.filter(pl.col('p_prod_max')>lower_limit)

In [57]:
# Filter for neighborhoods that as at least 1 plusskunde net exporting more than limit
def r5_filter_for_ami_net_export(df: pl.DataFrame, lower_limit: float):
    return df.filter(pl.col('net_export_max')>lower_limit)

In [58]:
# Filter for neighorhoods with aggregated net export ratio penetration
def r6_filter_for_neighborhood_net_export_to_peak_load(df: pl.DataFrame, lower_limit: float):
    print('hello')
    return df

In [59]:
# Scenario
def scenario(df: pl.DataFrame, scenario_args: dict) -> dict:
    df = r1_filter_penetration(df, args['r1_lb'])
    df = r2_filter_ami_cnt_lower_limit(df, args['r2_lb'])
    df = r3_filter_ami_prod_cnt_lower_limit(df, args['r3_lb'])
    df = r4_filter_include_for_production_over(df, args['r4_lb'])
    df = r5_filter_for_ami_net_export(df, args['r5_lb'])
    df = r6_filter_for_neighborhood_net_export_to_peak_load(df,  args['r6_lb'])

    # solve some statistics
    return {'number_of_neighborhoods'}

In [47]:
args = {'r1_lb':0, 'r2_lb':0, 'r3_lb':0, 'r4_lb':0, 'r5_lb':0, 'r6_lb':0}
scenario(df=df, scenario_args = args)

KeyboardInterrupt: 

In [60]:
r6_filter_for_neighborhood_net_export_to_peak_load(df,0)

hello


topology,coordinate,date_from,date_to,sample_cnt,ami_cnt,ami_load_cnt,ami_prod_cnt,res_pen_pers,p_load_max,p_prod_max,net_export_max,net_export_min,p_load_nb_max_24h_agg_kwh,p_prod_nb_max_24h_agg_kwh,p_gexp_nb_max_24h_agg_kwh
str,str,str,str,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64
"""S_645049_T_602…","""59.85786539754…","""2023-03-01T00:…","""2023-09-01T01:…",4417,1,1,0,0.0,110.0,0.0,0.0,-110.0,1680.0,0.0,0.0
"""S_2042_T_1306""","""59.23707291396…","""2023-03-01T00:…","""2023-09-01T01:…",79506,18,18,0,0.0,10.0,0.0,0.0,-10.0,967.9,0.0,0.0
"""S_1134873_T_11…","""60.43964784606…","""2023-03-01T00:…","""2023-09-01T01:…",4417,1,1,0,0.0,168.8,0.0,0.0,-168.8,2969.4,0.0,0.0
"""S_26_T_433028""","""59.23766985569…","""2023-03-01T00:…","""2023-09-01T01:…",30919,7,6,1,16.7,22.1,1.9,1.9,-22.1,813.0,9.5,0.0
"""S_730604_T_711…","""59.77068080945…","""2023-03-01T00:…","""2023-09-01T01:…",525623,119,119,5,4.2,50.6,9.8,9.8,-50.6,8677.0,255.9,0.0
"""S_17221_T_3415…","""59.25180506786…","""2023-03-01T00:…","""2023-09-01T01:…",13251,3,3,0,0.0,12.1,0.0,0.0,-12.1,230.9,0.0,0.0
"""S_502508_T_501…","""59.07820652361…","""2023-03-01T00:…","""2023-09-01T01:…",203182,46,46,2,4.3,20.3,12.8,12.8,-20.3,3915.5,130.3,0.0
"""S_7321_T_12192…","""59.25099546383…","""2023-03-01T00:…","""2023-09-01T01:…",75089,17,17,1,5.9,63.7,24.3,24.3,-63.7,4784.1,186.4,0.0
"""S_625694_T_604…","""59.24472638331…","""2023-03-01T00:…","""2023-09-01T01:…",17668,4,4,0,0.0,3.6,0.0,0.0,-3.6,129.6,0.0,0.0
